# Processing the final dataset



In [21]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"




Successfully saved authorization token.


In [2]:
export_distances = False
aggregate_LU = False

if (aggregate_LU):
    prefix = "aggregated"
else:
    prefix = "non_aggregated"

In [3]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")
proj = mature_biomass.projection().getInfo()

age_agbd = ee.Image(f"{data_folder}/age_agbd")
secondary = age_agbd.select("age")

if (export_distances):
    mature_biomass_vector = mature_biomass.reduceToVectors(geometry = secondary.geometry(), scale=500, maxPixels=1e13)
    distances_image = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.image.toAsset(
        image=distances_image,
        assetId = "projects/amazon-forest-regrowth/assets/distance_to_mature_biomass",
        description="distance_to_mature_biomass",
        region=secondary.geometry(),
        crs=proj['crs'],
        crsTransform=proj['transform'],
        maxPixels=1e13
    )
    task.start()



In [4]:

# Function to buffer and calculate mean
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(2000)))
    return buffered


In [39]:
categorical = ee.Image(f"{data_folder}/categorical")
distance = ee.Image(f"{data_folder}/distance_to_mature_biomass")

one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
biome_mask = categorical.select('biome').eq(1).Or(categorical.select('biome').eq(4))
combined_mask = one_hectare_mask.And(biome_mask)

categorical = categorical.updateMask(combined_mask)

dist_cat = distance.addBands(combined_mask)

dist_cat_sampled = dist_cat.stratifiedSample(
    numPoints = 1000000, classBand = 'one_hectare_mask', region = age_agbd.geometry(), geometries = True
)

dist_cat_sampled_buffered = dist_cat_sampled.map(buffer_distance)

nearest_mature = mature_biomass.reduceRegions(dist_cat_sampled_buffered, ee.Reducer.firstNonNull(), 
                                        crs = proj['crs'], crsTransform = proj['transform']
)

# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=nearest_mature,
    assetId = "projects/amazon-forest-regrowth/assets/nearest_mature1",
    description="nearest_mature"
)

task.start()

In [19]:
nearest_mature = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/nearest_mature")

nearest_mature = nearest_mature.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

# Convert the FeatureCollection to an image
nearest_mature_image = nearest_mature.reduceToImage(
    properties=['first'],
    reducer=ee.Reducer.first()
).toInt16().rename('nearest_mature')

export_task = ee.batch.Export.image.toAsset(
    image=nearest_mature_image,
    assetId = "projects/amazon-forest-regrowth/assets/nearest_mature_image",
    description='nearest_mature_export',
    crs=proj['crs'],
    crsTransform=proj['transform'],
    region = age_agbd.geometry(),
    maxPixels=1e13
)

export_task.start()

In [38]:

def export_csv(year):
    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = f"{prefix}_all"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{prefix}_{year}yr"

    nearest_mature = ee.Image(f"{data_folder}/nearest_mature_image").toInt16()
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")

    combined_mask = land_use_years.select("last_LU").And(nearest_mature)

    unified_data = (
        age_agbd.addBands(
            [distance, nearest_mature, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )

    unified_data_sampled = unified_data.stratifiedSample(
        numPoints = 10000, classBand = 'biome', region = age_agbd.geometry()
    )

    task = ee.batch.Export.table.toDrive(
        collection=unified_data_sampled, description=name
    )
    task.start()

# task = ee.batch.Export.table.toAsset(
#     collection=pts_bm, description=name, assetId=f"projects/amazon-forest-regrowth/assets/{name}"
# )
# task.start()

export_csv(0)
export_csv(5)
export_csv(10)
export_csv(15)

TypeError: Filter.neq() missing 1 required positional argument: 'value'

In [13]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')
br_shp = ee.FeatureCollection('projects/amazon-forest-regrowth/assets/raw/brasil_shp')

# Convert the FeatureCollection to an image
br_img = br_shp.reduceToImage(
    properties=['AREA_KM2'],  # Assuming 'first' is the current property name
    reducer=ee.Reducer.first()
).rename('br_img')

br_img = br_img.reproject(proj['crs'], proj['transform'])
br_img = br_img.gt(0)


south_american_countries = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
    'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

south_america = gaul.filter(ee.Filter.inList('ADM0_NAME', south_american_countries))

south_america_merged = south_america.union().geometry()

br_buffer = br_shp.geometry().buffer(-1000, maxError=5)
sa_buffer = south_america_merged.buffer(-1000, maxError=5)

remove_mask = br_buffer.symmetricDifference(sa_buffer)

# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=remove_mask,
    assetId = "projects/amazon-forest-regrowth/assets/remove_mask",
    description="remove_mask"
)
task.start()

# map = geemap.Map()
# map.addLayer(br_img, {}, "br_img")
# map.addLayer(brazil_buffer, {}, "brazil_buffer")
# # map.addLayer(south_america_merged, {}, "largest_polygon")
# map

In [57]:

# Convert the MultiPolygon to a FeatureCollection of individual polygons
geometry_fc = ee.FeatureCollection(south_america_merged.geometries().map(lambda geom: ee.Feature(ee.Geometry(geom))))

# Calculate the area of each polygon and add it as a property
area_fc = geometry_fc.map(lambda f: f.set('area', f.geometry().area()))

# Sort the polygons by area in descending order
sorted_fc = area_fc.sort('area', False)

# Get the largest polygon (the first feature in the sorted collection)
largest_polygon = ee.Feature(sorted_fc.first()).geometry()


In [14]:
# mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
# # mature_biomass_500m_countrywide = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")
# tst = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/nearest_mature")


# map = geemap.Map()
# map.addLayer(tst, {}, 'tst')
# map.addLayer(mature_biomass, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'mature_biomass')
# # map.addLayer(mature_biomass_100m, {"min": 0, "max": 400, "palette": ["blue", "red"]}, 'mature_biomass_100m')
# map